In [ ]:
import imaplib, email
from integrators.data.schema import Account, EmailMessage, MessageChannel
from integrators.pod.client import PodClient

class IMAPClient():
    
    def __init__(self, username, app_pw, host='imap.gmail.com', inbox='"[Gmail]/All Mail"'):
        self.client = imaplib.IMAP4_SSL(host)
        self.client.login(username, app_pw)
        # TODO: let user select mailbox
        self.client.select(inbox) # connect to inbox.
        
    def list_mailboxes(self):
        return self.client.list()
    
    def get_all_mail_uids(self):
        result, data = self.client.uid('search', None, "ALL") # search and return uids instead
        return data[0].split()
    
    def get_mail(self, uid):
        result, data = self.client.uid('fetch', uid, '(RFC822)')
        raw_email = data[0][1]
        return raw_email
    
    def get_all_mails(self, uids):
        res = []
        for uid in tqdm(uids):
            result, data = self.client.uid('fetch', uid, '(RFC822)')
            raw_email = data[0][1]
            res.append(raw_email)
        return res

    @staticmethod
    def part_to_str(part):
        bytes_ = part.get_payload(decode=True)
        charset = part.get_content_charset('iso-8859-1')
        chars = bytes_.decode(charset, 'replace')
        return chars
    
    def get_html(self, email_message_instance):
        maintype = email_message_instance.get_content_maintype()
        if maintype == 'multipart':

            parts = self._get_all_parts(email_message_instance)        
            res = None
            html_parts = [self.part_to_str(part) for part in parts if part.get_content_type() == "text/html"]
            if len(html_parts) > 0:
                if len(html_parts) > 1:
                    error_msg = "\n AND \n".join(html_parts)
                    print(f"WARNING: FOUND MULTIPLE HTML PARTS IN ONE MESSAGE {error_msg}")
                return html_parts[0]
            else:                
                return parts[0].get_payload()

        elif maintype == 'text':
            return email_message_instance.get_payload()
        
    def _get_all_parts(self, part):
        payload = part.get_payload()
        if isinstance(payload, list):
            return [x for p in payload for x in self._get_all_parts(p)]
        else:
            return [part]   
        
    def get_x_gm_thrid(self, uid):
        result, data = self.client.uid('fetch', uid, '(X-GM-THRID X-GM-MSGID)')
        return data[0].decode("utf-8").split(" ")[2]
    
    def decode_header(self, header):
        return str(email.header.make_header(email.header.decode_header(header)))

In [ ]:
def get_timestamp_from_message(message):
    date = message["date"]
    parsed_time = email.utils.parsedate(date)
    dt = email.utils.parsedate_to_datetime(date)
    timestamp = int(dt.timestamp() * 1000)
    
    return timestamp

def create_item_from_mail(gmail_id):
    mail = imap_client.get_mail(gmail_id)
    try:
        mail_utf8 = mail.decode("utf-8") 
    except (UnicodeDecodeError, AttributeError):
        print(f"Skipping email {external_id}, not utf-8 encoded")
        return
            
    message = email.message_from_string(mail_utf8)
                
    message_id = message["message-id"].replace("/", "")
    subject = message["subject"]
    timestamp = get_timestamp_from_message(message)
    
    from_name, from_mail = email.utils.parseaddr(message["from"])
    from_name = imap_client.decode_header(from_name)

    to_name, to_mail = email.utils.parseaddr(message["to"])
    to_name = imap_client.decode_header(to_name)

    reply_to_name, reply_to_mail = email.utils.parseaddr(message["reply-to"])
    reply_to_name = imap_client.decode_header(reply_to_name)

    content = imap_client.get_html(message)
    content = content.replace("=3D", "=")
    importJson = mail_utf8
        
    # TODO: is dateSent the right way to go? Might differ for whether you're sender or receiver
    # TODO: importJson
    email_item = EmailMessage(externalId=message_id, subject=subject, dateSent=timestamp, content=content)
    
    # Create Edges to accounts
    for address in [from_mail] if from_mail != "" else []:
        address_item = Account(externalId=address)
        email_item.add_edge('sender', address_item)
        
    for address in [to_mail] if to_mail != "" else []:
        address_item = Account(externalId=address)
        email_item.add_edge('receiver', address_item)
        
    for address in [reply_to_mail] if reply_to_mail != "" else []:
        address_item = Account(externalId=address)
        email_item.add_edge('replyTo', address_item)
        
    # Create edge to MessageChannel
    thread_id = imap_client.get_x_gm_thrid(gmail_id)
    message_channel = MessageChannel(externalId=thread_id)
    email_item.add_edge('messageChannel', message_channel)
    
    print(f'{email_item}')
    return email_item

In [ ]:
def download_mails(imap_client, gmail_ids, stop_at):
    all_mails = []
    
    # Download files
    for i, gmail_id in enumerate(gmail_ids):
        if stop_at is not None and i >= stop_at:
            print(f"stopped early at {stop_at}")
            break

        gmail_id = gmail_id.decode("utf-8")

        item = create_item_from_mail(gmail_id)
        all_mails.append(item)
    
    return all_mails

def merge_duplicate_items(all_mails):
    all_accounts = {}
    for email_item in all_mails:
        for edge in email_item.get_all_edges():
            account = edge.traverse(email_item)
            if not account.externalId in all_accounts:
                all_accounts[account.externalId] = account

    for email_item in all_mails:
        for edge in email_item.get_all_edges():
            edge.target = all_accounts[edge.target.externalId]
            
    return all_accounts

In [ ]:
# Store your credentials in this file:
file = open('tmp/credentials_gmail.txt','r')
imap_host = 'imap.gmail.com'
imap_user = file.readline()
imap_pw = file.readline()

imap_client = IMAPClient(username=imap_user, app_pw=imap_pw, host=imap_host)
gmail_ids = imap_client.get_all_mail_uids()
all_mails = download_mails(imap_client, gmail_ids, 10)

# Merge email accounts/messageChannels here
# TODO: create better way to do this
all_accounts = merge_duplicate_items(all_mails)
print(all_accounts)

pod_client = PodClient()
pod_client.delete_all()

# Create all email and account items
all_thread_ids = set()
for email_item in all_mails:
    pod_client.create(email_item)
    all_thread_ids.add(email_item.messageChannel[0].externalId)
for (external_id, item) in all_accounts.items():
    pod_client.create(item)
        
# Create all edges from emails to accounts
for email_item in all_mails:
    pod_client.create_edges(email_item.get_all_edges())

# TODO: create email threads


In [ ]:
# All emails are now in the pod, create their thread edges here:
# for thread_id in all_thread_ids:
#     pod_client.search_by_fields({"":""})